In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [2]:
#For each camera, we extract the videos from the mat files
Mat1 = scipy.io.loadmat('/Users/nadaelarabi/MATH-494/M494H2/cam1_1.mat')
dataMat1 = Mat1["vidFrames1_1"]

In [3]:
Mat2 = scipy.io.loadmat('/Users/nadaelarabi/MATH-494/M494H2/cam2_1.mat')
dataMat2 = Mat2["vidFrames2_1"]

In [4]:
Mat3 = scipy.io.loadmat('/Users/nadaelarabi/MATH-494/M494H2/cam3_1.mat')
dataMat3 = Mat3["vidFrames3_1"]

In [5]:
#We need to determine the dimensions of each of the camera to see what dimensions our data matrix should have
print("The dimensions of cam1's take are: ",dataMat1.shape," cam2's: ",dataMat2.shape," cam3's: ",dataMat3.shape)

The dimensions of cam1's take are:  (480, 640, 3, 226)  cam2's:  (480, 640, 3, 284)  cam3's:  (480, 640, 3, 232)


In [6]:
#We write the videos necessary to capture the motion from each camera
#We only need to change the dataMati and the name of the videos for each camera take, I wrote them out as 
#camera1, camera2, camera3
#Sometimes necessary to first write the filename as mp4 inside 'filename'
height, width,_,nbFrames = dataMat3.shape
codec_id = "mp4v" # ID for a video codec.
fourcc = cv.VideoWriter_fourcc(*codec_id)
filename = "/Users/nadaelarabi/MATH-494/M494H2/camera3.mp4v"
out = cv.VideoWriter(filename, fourcc=fourcc, fps=20, frameSize=(width, height))


for i in range(nbFrames):
    out.write(dataMat3[:,:,:,i])


In [223]:
#Here we declare the matrix that will store our x and y coordinates
x_y_coordinateMatrix = np.zeros((0,225))

In [219]:
#Now we can set up the motion tracking 

#First, we begin by setting up a dictionary of legacy motion trackers in opencv
TrDict = {
    'csrt' : cv.legacy.TrackerCSRT_create,
    'mosse' : cv.legacy.TrackerMOSSE_create,
    'kcf' : cv.legacy.TrackerKCF_create,
    'medianflow': cv.legacy.TrackerMedianFlow_create,
    'mil' : cv.legacy.TrackerMIL_create,
    'tld' : cv.legacy.TrackerTLD_create,
    'boosting' : cv.legacy.TrackerBoosting_create
}

In [258]:
#Now we initialize the tracker, csrt was chosen after testing with the other trackers and was found to be the best
tracker = TrDict['csrt']()

In [259]:
#The code below allows us to manually draw a rectangle that will track the motion of the bucket
#Recommended to draw a larger box as it makes it less susceptible to be thown off 

v = cv.VideoCapture('camera3.mp4')
ret, frame = v.read()
cv.imshow('Frame',frame)
bb = cv.selectROI('Frame',frame)
tracker.init(frame,bb)
    

True

In [260]:
#Displaying and storing the coordinates into arrays
x_coordinates = np.zeros((284,1))
y_coordinates = np.zeros((284,1))
index = 0
while True:
    ret,frame = v.read()
    if not ret:
        break
    (success,box) = tracker.update(frame)
    if success:
        (x,y,w,h) = [int(a) for a in box]
        cv.rectangle(frame,(x,y),(x+w,y+h),(250,0,250), 2)
        x_centerCoordinate = (x+w)/2
        y_centerCoordinate = (y+h)/2
        print(x_centerCoordinate,y_centerCoordinate)
        x_coordinates[index] = x_centerCoordinate
        y_coordinates[index] = y_centerCoordinate
    cv.imshow('Frame',frame)
    key = cv.waitKey(30)
    index = index+1
    if key == ord('q'):
        break

v.release()
cv.destroyAllWindows()

211.0 167.5
204.5 166.0
204.0 166.0
199.5 165.0
199.0 165.0
195.0 164.5
192.5 163.0
193.0 164.0
192.5 163.0
193.0 164.0
195.0 164.0
195.0 164.0
199.0 164.0
206.0 164.5
206.5 164.5
213.5 165.5
213.5 165.5
222.0 166.5
222.0 166.5
231.5 167.5
240.0 168.0
240.5 169.0
246.5 170.0
246.0 169.5
251.0 170.5
252.0 171.0
257.0 172.0
257.0 173.0
257.0 172.5
257.5 173.5
257.0 174.0
254.0 173.0
254.0 173.0
249.0 172.5
240.0 170.0
241.0 170.5
232.0 168.0
233.0 169.0
224.0 167.5
224.5 167.5
214.5 164.0
209.0 164.5
209.0 164.5
203.5 163.0
203.5 163.5
199.5 162.0
199.5 162.5
198.5 163.0
198.5 162.5
198.5 162.5
201.5 163.5
201.0 164.0
205.5 164.0
205.5 164.0
210.5 164.5
219.0 166.0
219.0 166.0
226.5 167.0
226.5 167.5
236.0 169.0
236.5 169.5
243.0 170.5
249.0 171.5
249.0 172.5
253.5 172.5
254.0 173.5
256.0 172.5
256.5 173.0
255.5 172.0
253.0 171.5
253.5 172.0
250.0 171.5
250.0 171.5
245.0 170.5
245.0 170.5
238.5 169.5
230.5 168.0
230.0 168.0
220.5 166.5
219.0 165.0
212.0 165.0
211.0 164.0
204.5 163.5
199.

In [261]:
#Here we will resize our coordinate arrays so that they are an appropriate shape
#Since the camera take with the fewest number of frames generates 225 data points, 
x_coordinates = np.resize(x_coordinates,(225,1))
y_coordinates = np.resize(y_coordinates,(225,1))

In [262]:
#Now we can reshape the coordinates to make them one long row
x_coordinates = np.reshape(x_coordinates,(1,225))
y_coordinates = np.reshape(y_coordinates,(1,225))

In [263]:
np.shape(x_y_coordinateMatrix)

(4, 225)

In [264]:
#x_y_coordinateMatrix.append(x_coordinates)
x_y_coordinateMatrix = np.append(x_y_coordinateMatrix,x_coordinates,axis=0)
#x_y_coordinateMatrix.append(y_coordinates)


In [265]:
x_y_coordinateMatrix = np.append(x_y_coordinateMatrix,y_coordinates,axis=0)

In [266]:
np.shape(x_y_coordinateMatrix)

(6, 225)

In [267]:
x_y_coordinateMatrix

array([[207.5, 207.5, 207.5, ..., 204. , 204. , 203.5],
       [170.5, 170.5, 170.5, ..., 219.5, 219.5, 224.5],
       [183.5, 186. , 186. , ..., 190. , 190. , 189.5],
       [200.5, 201.5, 196.5, ..., 131.5, 138. , 138.5],
       [211. , 204.5, 204. , ..., 243.5, 248.5, 248.5],
       [167.5, 166. , 166. , ..., 167.5, 168. , 168. ]])

In [268]:
np.save('x_y_coordinateMatrix',x_y_coordinateMatrix)

In [ ]:
plt.plot()